In [1]:
import zipfile
import os, shutil
import collections
from pathlib import Path
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.regularizers import l2
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import pylab as pl
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import itertools
import math


# audio lib
import librosa
import librosa.display
from librosa.util import fix_length

import IPython.display as ipd
from PIL import Image

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
output_folder = "/kaggle/working/melspectrogram_tmp/"
os.mkdir(output_folder)

# Classificazione Canzoni Utente

Una volta generato il modello si andrà a utilizzarlo per fare inferenza sul mood delle canzoni dell'utente. In questo caso si utilizzano come test canzoni scelte in maniera da coprire più generi e mood possibili. Il sistema andrà a generare per ogni canzone, divisa in blocchi di 6 secondi, vari spectrogrammi di Mel e si andrà a classificare ognuno di essi, arrivando ad avere per ogni canzone una percentuale del tipo di mood evocato, andando quindi a considerare in maniera più fine il mood totale della canzone.

In [4]:
model = load_model('../input/effnetb3spectrogram/weights.cnn.hdf5')

2022-05-26 16:32:57.792230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 16:32:57.914654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 16:32:57.915477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 16:32:57.916773: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
data = {
    'ID' : [],
    'A-V-' : [],
    'A-V+' : [],
    'A+V-' : [],
    'A+V+' : []
}

directory = '../input/my-playlist/Playlist/'

SAMPLE_RATE = 22050
TRACK_DURATION = 480
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

num_segments = 80
samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)


num_mel = 96
n_fft = 2048
hop_length = 512

num_mel_vectors_per_segment = math.ceil(samples_per_segment / hop_length)


In [6]:
print("Classificazione mood canzioni in corso...")
print("Legenda:")
print("0: basso Arousal - basso valence ")
print("1: basso Arousal - alto valence ")
print("2: alto Arousal - basso valence ")
print("3: alto Arousal - alto valence ")
print("----------------------------------------------------")

# per ogni file musicale genero uno spectogramma temporaneo
for filename in os.listdir(directory):
    labels = []
    
    filename_no_extension = Path(filename).stem
    data["ID"].append(filename_no_extension)
    
    file_path = os.path.join(directory, filename)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE,  offset=5.0, duration=TRACK_DURATION)

    # genero spectogrammi per ogni 6 secondi di canzone
    for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mel
                    mel = librosa.feature.melspectrogram(y=signal[start:finish], sr=sample_rate, n_mels=num_mel, hop_length=hop_length)
                    mel = mel.T

                    
                    if (len(mel) == num_mel_vectors_per_segment):
                        
                        
                        img_name = str(d) + '.png'

                        


                        fig = plt.figure(figsize=(3, 3), dpi=100)
                        S_dB = librosa.power_to_db(mel.T)
                        img = librosa.display.specshow(S_dB)
                        
                        plt.savefig("{}{}".format(output_folder,img_name))
                        plt.close(fig)

    # per ogni spectrogramma generato dalla canzone eseguo la predizione e creo una lista con tutte le predizioni riferite ad una canzone
    for i in os.listdir(output_folder):
        
        file_path = os.path.join(output_folder, i)
        pic = image.load_img(file_path)
        pic = np.expand_dims(np.array(pic), axis=0)
        classification = model.predict(pic)
        labels.append(np.argmax(classification))
    
    
    # creo una lista con le percentuali di mood per ogni canzone

    counter = collections.Counter(labels)
    most_common= [(i, round(counter[i] / len(labels) * 100.0, 1)) for i, count in counter.most_common()]
    
    # trasformo la lista in campi per il dataframe
    i0 = False
    i1 = False
    i2 = False
    i3 = False
    
    for i in range(len(most_common)):
        if most_common[i][0] == 0:
            i0 = True
            data['A-V-'].append(most_common[i][1])
        elif most_common[i][0] == 1:
            i1 = True
            data['A-V+'].append(most_common[i][1])
        elif most_common[i][0] == 2:
            i2 = True
            data['A+V-'].append(most_common[i][1])
        elif most_common[i][0] == 3:
            i3 = True
            data['A+V+'].append(most_common[i][1])
            
    
    if not(i0):
        data['A-V-'].append(0)
    if not(i1):
        data['A-V+'].append(0)
    if not(i2):
        data['A+V-'].append(0)
    if not(i3):
        data['A+V+'].append(0)

            
        
    

    print("File: "+filename)
    print("Predizioni:")
    print(labels)
    print("Mood in percentuale: "+ str(most_common))    
    print("----------------------------------------------------")
    
    # cancello la directory temporanea e la ricero
    shutil.rmtree(output_folder)
    os.mkdir(output_folder)

Classificazione mood canzioni in corso...
Legenda:
0: basso Arousal - basso valence 
1: basso Arousal - alto valence 
2: alto Arousal - basso valence 
3: alto Arousal - alto valence 
----------------------------------------------------


2022-05-26 16:33:20.181203: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-26 16:33:22.926841: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


File: Fabrizio De Andr - La canzone di Marinella.mp3
Predizioni:
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mood in percentuale: [(0, 96.8), (1, 3.2)]
----------------------------------------------------
File: John Coltrane - Naima.mp3
Predizioni:
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mood in percentuale: [(0, 88.1), (1, 11.9)]
----------------------------------------------------
File: Nirvana - Come As You Are.mp3
Predizioni:
[2, 2, 3, 2, 3, 3, 3, 2, 0, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 0, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2, 2]
Mood in percentuale: [(2, 47.2), (3, 47.2), (0, 5.6)]
----------------------------------------------------
File: Queen - Radio Ga Ga.mp3
Predizioni:
[2, 2, 3, 3, 3, 1, 3, 0, 3, 0, 1, 3, 3, 3, 1, 1, 3, 3, 3, 1, 3, 3, 1, 2, 1, 3, 3, 3, 3, 1, 2, 3, 0, 3, 1, 1, 3, 3, 0, 0, 2, 2, 3, 0, 3, 0, 0, 3, 3, 1, 3, 3, 3, 2, 3, 2, 3]

In [7]:
df = pd.DataFrame(data)
df.to_csv('mel.csv',index=False)

df

,ID,A-V-,A-V+,A+V-,A+V+
0,Fabrizio De Andr - La canzone di Marinella,96.8,3.2,0.0,0.0
1,John Coltrane - Naima,88.1,11.9,0.0,0.0
2,Nirvana - Come As You Are,5.6,0.0,47.2,47.2
3,Queen - Radio Ga Ga,14.0,19.3,14.0,52.6
4,Red Hot Chili Peppers - By The Way,22.9,5.7,25.7,45.7
5,Imagine Dragons - On Top of the World,6.7,13.3,6.7,73.3
6,Green Day - Time of Your Life,50.0,29.2,0.0,20.8
7,Oasis - Wonderwall,8.1,0.0,10.8,81.1
8,Eagles - Hotel California,29.7,14.1,1.6,54.7
9,Red Hot Chili Peppers - Snow,47.8,6.5,13.0,32.6
